## Classifying hate speech

This is our text classification model.
The text is converted to n-grams with TF-IDF scores.
The number of n-grams and the number of features to include can be changed.
The statistical models to be used are Naïve Bayes, Linear Support Vector Models and Random Forest

The model is created trough a pipeline. In this way, we can access the non-vectorized texts

In [ ]:
# Import things we need

#!pip install nltk
#!pip install pandas

import numpy as np
import nltk
from numpy.random import RandomState
import pickle
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

## Load and split data

In [ ]:
# Load data as a DataFrame. 

# Our data is ordered like this: "#1This is a hate speech text \n #2This is not hate speech \n"
# In this way we get a column with 'Type' and a column with 'Entry'
# We split at "#" to get type 1 and 2 (HS and non-HS)

data_hs = pd.read_csv('data.txt', delimiter = "#", header = None, names = ["Type", "Entry"])

data_hs = data_hs.dropna() #drop the NaN's

#display(data_hs)

In [ ]:
# Split the data into train and test set
# We set a random_state so that the data is split the same way everytime we run this code
# Test_size = 0.2 means that we have 80% training and 20% testing

X_train, X_test, y_train, y_test = train_test_split(data_hs['Entry'], data_hs['Type'], random_state = 10, test_size=0.2)

## Create and test model

In [ ]:
#Create the model

# ngram_range defines that we want unigrams, bigrams and trigrams.
# max_features means that we want the n best features. 
# min_df = n means that features are only chosen if they are in n or more documents. 
# max_df = 0.7 means that we only take features that occur in 70% of the texts. (This is not included)
# All of the above features can be changed

# Choose the statistical model you want

pipeline_1 = Pipeline([('vect', TfidfVectorizer(ngram_range=(1, 3), max_features=10000, min_df=5)), 
                     ('clf', MultinomialNB())]) 
                     #('clf', LinearSVC())]) 
                     #('clf', RandomForestClassifier())]) 

# Fit the model
model = pipeline_1.fit(X_train, y_train)

# Print accuracy score
print("accuracy score: " + str(model.score(X_test, y_test)))

In [ ]:
# Get precision, recall and F1

y_pred = model.predict(X_test)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

## Most predictive features

In [ ]:
# Prepare data for getting the best features - access the unvectorized n-grams

vectorizer = model.named_steps['vect']
clf = model.named_steps['clf']

feature_names = vectorizer.get_feature_names()
feature_names = np.asarray(feature_names)

In [ ]:
# Get most predictive features

N = 15 # How many features do you want?

HS_list = []
NONHS_list = []

# Most predictive for hate speech
HStop = np.argsort(clf.coef_[0])[:N]
print('Best feature names:', ', '.join(feature_names[HStop]))
for i in feature_names[HStop]:
    HS_list.append(i)

# Least predictive features for hate speech (not so interesting though)
#NONHStop = np.argsort(clf.coef_[0])[-N:]
#print('\nBest feature names:', ', '.join(feature_names[NONHStop]))
#for i in feature_names[NONHStop]:
#    NONHS_list.append(i)
    
#Make a dataframe
df = pd.DataFrame()
df['Hate speech']  = HS_list
#df['Non-hate speech'] = NONHS_list

display(df)

## Show misclassifications

In [ ]:
# Show all misclassifications

misclass = []

for item, label in zip(X_test, y_test):
    result = model.predict([item])
    if result != label:
        mis = ("PREDICTED = %s, TRUE = %s" % (result, label), item)
        misclass.append(mis)
        
#print(misclass)

In [ ]:
#Sort misclassifications into two lists

mis_HS = []
mis_NONHS = []

for item, label in zip(X_test, y_test):
    result = model.predict([item])
    if result != label:
        if result == 1:
            mis_HS.append(item)
        if result == 2:
            mis_NONHS.append(item)

print(len(mis_HS))
print(len(mis_NONHS))

In [ ]:
#Save the two lists as .txt files

with open('misclass_as_HS.txt','w') as output:
    output.write('MISCLASSIFIED AS HATE SPEECH:\n')
    for line in mis_HS:
        output.write(line)
        output.write('\n')

with open('misclass_as_NONHS.txt','w') as output:
    output.write('MISCLASSIFIED AS NON-HATE SPEECH:\n')
    for line in mis_NONHS:
        output.write(line)
        output.write('\n')

print('Done!')

## Try model, save model

In [ ]:
# Here we can try the model

test = input('Write a sentence: \n')

if model.predict([test]) == 1:
    print('Buuh! Hate speech!')
else:
    print('So nice! Have a great day!')

In [ ]:
# Saving the model

with open('classifier', 'wb') as picklefile:
    pickle.dump(model,picklefile)
    
# To open it later:
#with open('classifier_SVM', 'rb') as training_model:
#    model = pickle.load(training_model)